In [1]:
import sys
sys.path.insert(0, '/home/jonathan/Documents/mi3-balance/build/python')

import drjit as dr
import mitsuba as mi
from drjit.auto import Float, UInt
import time

mi.set_variant('cuda_ad_rgb')

In [2]:
from furnace_scene import make_scene, visualize_scene

color_ref = [0.2, 0.25, 0.7]
scene = make_scene(color_ref)

In [3]:
# image = mi.render(scene)
# print(image)

# import matplotlib.pyplot as plt
# plt.imshow(image)

In [4]:
from radiosity import SceneSurfaceSampler, RadianceCacheMITSUBA, compute_loss

NUM_WI_DIRECTIONS = 16
SAMPLES_PER_RAY_LI = 32
SAMPLES_PER_RAY_LO = 32
NUM_GEO_SAMPLES = 2

scene_sampler = SceneSurfaceSampler(scene)
radiance_cache = RadianceCacheMITSUBA(scene)

# Initialize BSDF to train
bsdf_train = mi.load_dict({
    "type": "diffuse",
    "reflectance": {
        "type": "rgb",
        "value": [0.5, 0.5, 0.5]
    }
})

params = mi.traverse(bsdf_train)
opt = mi.ad.Adam(lr=0.05)
key = "reflectance.value"
dr.enable_grad(params[key])
opt[key] = params[key]
params.update(opt)

for it in range(100):

    # if True: ## Temp workaround
    #     # Initialize samplers
    #     scalar_sampler = mi.load_dict({
    #                 'type': 'independent',
    #                 'sample_count': 1
    #             })
    #     scalar_sampler.seed(it, 1)

    time1 = time.time()

    # Evaluate the objective function for the current BSDF params
    loss = compute_loss(scene_sampler, radiance_cache, bsdf_train, 
        NUM_GEO_SAMPLES, NUM_WI_DIRECTIONS, 
        SAMPLES_PER_RAY_LO, SAMPLES_PER_RAY_LI)

    # Backpropagate through the rendering process
    dr.backward(loss)

    # Optimizer: take a gradient descent step
    opt.step()

    # Post-process the optimized parameters to ensure legal color values.
    opt[key] = dr.clip(opt[key], 0.0, 1.0)

    # Update the scene state to the new optimized values
    params.update(opt)

    # losses.append(dr.detach(loss))
    time2 = time.time()
    print(f"Iteration {1+it:03d} [{time2 - time1:2f}]: Loss = {loss}, p = {opt[key]}")


Iteration 001 [0.034878]: Loss = [0.00965773], p = [[0.450009, 0.450006, 0.55]]
Iteration 002 [0.010365]: Loss = [0.00603595], p = [[0.400506, 0.400708, 0.599355]]
Iteration 003 [0.009888]: Loss = [0.00329287], p = [[0.352004, 0.352914, 0.647336]]
Iteration 004 [0.009769]: Loss = [0.00141846], p = [[0.305207, 0.307807, 0.69289]]
Iteration 005 [0.009837]: Loss = [0.000361694], p = [[0.261043, 0.266973, 0.734604]]
Iteration 006 [0.010510]: Loss = [1.15834e-05], p = [[0.220654, 0.232245, 0.770807]]
Iteration 007 [0.010019]: Loss = [0.000186456], p = [[0.185296, 0.205262, 0.799903]]
Iteration 008 [0.009774]: Loss = [0.000649929], p = [[0.156137, 0.186941, 0.820828]]
Iteration 009 [0.009795]: Loss = [0.00116204], p = [[0.133997, 0.177237, 0.833351]]
Iteration 010 [0.009800]: Loss = [0.00154144], p = [[0.119176, 0.175328, 0.838017]]
Iteration 011 [0.009832]: Loss = [0.00169898], p = [[0.111439, 0.179991, 0.835856]]
Iteration 012 [0.009796]: Loss = [0.00163037], p = [[0.110145, 0.189885, 0.82

In [5]:
print(bsdf_train)
print(color_ref)

SmoothDiffuse[
  reflectance = SRGBReflectanceSpectrum[
    value = [[0.219635, 0.272802, 0.738692]]
  ]
]
[0.2, 0.25, 0.7]
